In [1]:
import json
import time 

from kafka import KafkaProducer



In [2]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

In [3]:
server = 'localhost:9093'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [ ]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    ts=time.time()
    producer.send(topic_name, value=message)
    tf=time.time()
    print(f'took {(tf - ts):.6f} seconds')
    print(f"Sent: {message}")
    time.sleep(0.05)
    print('next \n')
print('done sending flush next')
    
ts=time.time()
producer.flush()
tf=time.time()
print(f'took {(tf - ts):.6f} seconds')

t1 = time.time()
print(f'took {(t1 - t0):.20f} seconds')

In [ ]:
#!docker exec redpanda-1 rpk topic consume test-topic


In [ ]:
#!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz

In [ ]:
#!gunzip green_tripdata_2019-10.csv.gz


In [4]:
import pandas as pd

# Specify the columns you want to read
columns = [
    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount'
]

# Read the specified columns from the compressed CSV
df_green = pd.read_csv('green_tripdata_2019-10.csv.gz', usecols=columns)

# Iterate over the records in the dataframe
for row in df_green.itertuples(index=False):
    # Convert the row to a dictionary
    row_dict = {col: getattr(row, col) for col in row._fields}
    
    # Print the dictionary
    print(row_dict)
    
    # Break after the first iteration for demonstration purposes
    break

    # TODO: implement your logic for sending the data here
    # For example, producer.send('your_topic_name', value=row_dict)


{'lpep_pickup_datetime': '2019-10-01 00:26:02', 'lpep_dropoff_datetime': '2019-10-01 00:39:58', 'PULocationID': 112, 'DOLocationID': 196, 'passenger_count': 1.0, 'trip_distance': 5.88, 'tip_amount': 0.0}


In [5]:
import time

topic_name = 'green-trips'

t0 = time.time()

for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(topic_name, value=row_dict)

producer.flush()  # Ensure all messages are sent

t1 = time.time()
duration = t1 - t0
print(f"Time taken to send the data: {duration} seconds")


Time taken to send the data: 63.10613560676575 seconds


In [6]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/okibaba/spark/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/okibaba/.ivy2/cache
The jars for the packages stored in: /home/okibaba/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dcff2b39-be91-49fb-8a65-3c5dfad1c2ac;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downl

24/04/08 02:44:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9093") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [8]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/04/08 03:06:18 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3739a923-c840-46ae-8edc-e926ba35cd03. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/08 03:06:18 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 01:53:53", "lpep_dropoff_datetime": "2019-10-01 02:36:09", "PULocationID": 181, "DOLocationID": 216, "passenger_count": 1.0, "trip_distance": 8.66, "tip_amount": 0.0}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 4, 8, 2, 24, 14, 735000), timestampType=0)


In [9]:
query.stop()

In [10]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [11]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [12]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/04/08 03:11:31 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3f5f37db-2caa-4f11-ab77-6df139d4f082. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/08 03:11:31 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
Row(lpep_pickup_datetime='2019-10-01 01:53:53', lpep_dropoff_datetime='2019-10-01 02:36:09', PULocationID=181, DOLocationID=216, passenger_count=1.0, trip_distance=8.66, tip_amount=0.0)


In [13]:
query.stop()

In [ ]:
from pyspark.sql import SparkSession, functions as F

# Initialize Spark session (assuming it's not already initialized)
#spark = SparkSession.builder.appName("PopularDestinations").getOrCreate()

# Assuming green_stream is your streaming DataFrame with the necessary data loaded
# Add a timestamp column to the DataFrame
green_stream = green_stream.withColumn("timestamp2", F.current_timestamp())

# Group by a 5-minute window and DOLocationID, and count occurrences
popular_destinations = green_stream2.groupBy(
    F.window(F.col("timestamp2"), "5 minutes"),
    "DOLocationID"
).count()

# Order by count in descending order to find the most popular destinations
popular_destinations = popular_destinations.orderBy(F.desc("count"))

# Output the results to the console in a streaming fashion
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()


In [ ]:
from pyspark.sql import functions as F

popular_destinations = green_stream \
    .withColumn("timestamp", F.current_timestamp()) \
    .groupBy(F.window(F.col("timestamp"), "5 minutes"), F.col("DOLocationID")) \
    .count() \
    .orderBy(F.col("count").desc())

query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

24/04/08 03:21:00 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-71062b0a-9ef1-47c2-b796-b9fc2afc95d9. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/08 03:21:00 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------------+------+
|window                                    |DOLocationID|count |
+------------------------------------------+------------+------+
|{2024-04-08 03:20:00, 2024-04-08 03:25:00}|74          |177402|
|{2024-04-08 03:20:00, 2024-04-08 03:25:00}|42          |159408|
|{2024-04-08 03:20:00, 2024-04-08 03:25:00}|41          |140604|
|{2024-04-08 03:20:00, 2024-04-08 03:25:00}|75          |128396|
|{2024-04-08 03:20:00, 2024-04-08 03:25:00}|129         |119287|
|{2024-04-08 03:20:00, 2024-04-08 03:25:00}|7           |115322|
|{2024-04-08 03:20:00, 2024-04-08 03:25:00}|166         |108448|
|{2024-04-08 03:20:00, 2024-04-08 03:25:00}|236         |79129 |
|{2024-04-08 03:20:00, 2024-04-08 03:25:00}|223         |75415 |
|{2024-04-08 03:20:00, 2024-04-08 03:25:00}|238         |73180 |
|{2024-04-08 03:20:00, 2024-04-08 03:25:00}|82          |7

In [ ]:
query.stop()
